### We will be using pytorch datasets and data loaders to implement  residual U-net model 

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, sampler, Subset, WeightedRandomSampler


from torchvision import transforms, datasets

import logging
import numpy as np
from utils import *


import time
import copy

import skimage 
from skimage import io 
import logging
from PIL import Image

import os 
from os.path import splitext
from os import listdir

import numpy as np
import pandas as pd
from pandas import DataFrame

from torchvision.transforms import Compose, ToTensor, Resize, ToPILImage
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000  # incase PIL gives error

In [5]:
root_dir = '/home/abharani/cs231n_project'
data_dir = '/home/abharani/cs231n_project/data'
dir_checkpoint = os.path.join(root_dir, 'checkpoints')


## Dataset I 
Created from Image Folder and performing save operation for every image it pre-processes. 

In one epoch, it will save all the images and then we can use the saved images later to actual training. This will save pre-processing image everytime

In [6]:
# import torch
# from torchvision import datasets


# class ImageFolderWithPaths(datasets.ImageFolder):
#     """Custom dataset that includes image file paths. Extends
#     torchvision.datasets.ImageFolder
#     """
       
#     @classmethod
#     def preprocess(cls, image):
#         WINDOW_SIZE = 128
#         STRIDE = 64
#         K = 16
        
#         image, best_coordinates, best_regions = generate_patches(image, window_size=WINDOW_SIZE, stride=STRIDE, k=K)
#         glued_image = glue_to_one_picture(best_regions, window_size=WINDOW_SIZE, k=K)
        
#         return glued_image           
    
#     def __getitem__(self, index):
#         # this is what ImageFolder normally returns 
#         original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)

#         image, isup_grade = np.array(original_tuple[0]), original_tuple[1]
#         image = self.preprocess(image)
#         PIL_image = Image.fromarray((image * 255).astype(np.uint8))

#         # Let's get the image file path and extract file_name
#         path = self.imgs[index][0]
#         file_name = path.split("/")[-1].replace(".jpg", "_g.jpg")
#         PIL_image.save(file_name)
        
#         # Apply Tranformation
#         transform=Compose([Resize((224,224)),ToTensor(), 
#                            Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])])
        
#         image = transform(PIL_image)

#         return {'image': image, 'isup_grade' : isup_grade}  

    
# # EXAMPLE USAGE:
# # instantiate the dataset and dataloader

# # dataset_trial = ImageFolderWithPaths('/home/abharani/data/train_images') # our custom dataset
# # dataloader_trial = DataLoader(dataset_trial)

# # # iterate over data
# # for inputs, labels, paths in dataloader_trial:
# #     # use the above variables freely
# #     print(inputs['image'].shape, labels, paths)
# #     break 

# # sample = dataset_trial[0]
# # print(sample.keys())
# # print(sample['image'].shape, sample['isup_grade'])

## Dataset II
Created from ImageFolder on big size .tiff files. This dataset class does not include save operation

In [7]:

# class BiopsyDataClass(Dataset):
    
#     def __init__(self, image_path, transform=None):
#         super(BiopsyDataClass, self).__init__()
#         self.data = datasets.ImageFolder(image_path)    # Create data from folder
        
#         self.transform = transform
        
#     @classmethod
#     def preprocess(cls, image):
#         WINDOW_SIZE = 128
#         STRIDE = 64
#         K = 16
        
#         image, best_coordinates, best_regions = generate_patches(image, window_size=WINDOW_SIZE, stride=STRIDE, k=K)
#         glued_image = glue_to_one_picture(best_regions, window_size=WINDOW_SIZE, k=K)
        
#         return glued_image        
        
#     def __getitem__(self, idx):

#         image, isup_grade = self.data[idx]
#         image = np.array(image)
#         image = self.preprocess(image)
#         PIL_image = Image.fromarray((image * 255).astype(np.uint8))
        
#         if self.transform:
#             image = self.transform(PIL_image) 
# #             Image.save()
# #             print(PIL_image.type)
#         return {'image': image, 'isup_grade' : isup_grade}        
    
#     def __len__(self):
#         return len(self.data)

## Dataset III
Created from csv and then do a lookup for image_id inside train_images folder. Before using this filter the csv to exclude the image_id not present in train_images folder.

In [8]:
# class BasicDataset(Dataset):
#     def __init__(self, csv_file, data_dir, transform=None):
#         """
#         Args:
#             csv_file (string): Path to the csv file with annotations.
#             root_dir (string): Directory with all the images.
#             transform (callable, optional): Optional transform to be applied
#                 on a sample.
#         """        
#         self.data_frame = pd.read_csv(csv_file)

#         ## Local images only  
        
#         # local_files_df = DataFrame([os.path.splitext(filename)[0] for filename in os.listdir('/Users/abharani/Documents/myworkspace/cs231n_project/data/train_images/') if filename.endswith(".tiff")], columns=["image_id"])
#         # self.data_frame = local_files_df.join(data_frame.set_index('image_id'), on='image_id')
#         # ##

#         self.data_dir = data_dir
#         self.transform = transform
#         logging.info('Creating dataset with {} examples'.format(len(self.data_frame)))
        
#     def __len__(self):
#         return len(self.data_frame)

#     @classmethod
#     def preprocess(cls, image):
#         WINDOW_SIZE = 128
#         STRIDE = 64
#         K = 16
        
#         image, best_coordinates, best_regions = generate_patches(image, window_size=WINDOW_SIZE, stride=STRIDE, k=K)
#         glued_image = glue_to_one_picture(best_regions, window_size=WINDOW_SIZE, k=K)
        
#         return glued_image


#     def __getitem__(self, idx):
        
#         image_id = self.data_frame.iloc[idx]['image_id']
#         image_file_path = os.path.join(os.path.join(self.data_dir,'train_images'),image_id + ".tiff")
#         image = skimage.io.MultiImage(image_file_path)[-1]
#         image = np.array(image)

#         isup_grade = self.data_frame.iloc[idx]['isup_grade']

#         image = self.preprocess(image)
#         PIL_image = Image.fromarray((image * 255).astype(np.uint8))
# #         print("Image type {}".format(image.type)) #numpy.ndarray
# #         print(PIL_image.type)                     # Image object      

        
#         if self.transform:
            
#             image = self.transform(PIL_image)
        
#         return {'image': image, 'isup_grade' : isup_grade}




#### Step1 : 
Afrer saving the images generated from Dataset class ImageFolderWithPaths. Create dataset using the ImageFolder.


Since Data is imbalanced, performing Weighted Random Sampling not Random Sampling.



In [11]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader


def train_val_dataset(dataset, val_split=0.25, generate_small=False):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets_created = {}
    if generate_small:
        print("Generating Small Train, Test Dataset")
        datasets_created['train'] = Subset(dataset, train_idx[0:200])
        datasets_created['test'] = Subset(dataset, val_idx[0:50])
    else:
        datasets_created['train'] = Subset(dataset, train_idx)
        datasets_created['test'] = Subset(dataset, val_idx)        
    return datasets_created

dataset = ImageFolder('/home/abharani/cs231n_project/glued_images/data', 
                      transform=Compose([Resize((224,224)),ToTensor(), 
                                        Normalize(mean=[0.485, 0.456, 0.406],
                                                   std=[0.229, 0.224, 0.225])]))

# print(len(dataset))
# datasets = train_val_dataset(dataset,generate_small=False)
# print(len(datasets['train']))
# print(len(datasets['val']))
# # The original dataset is available in the Subset class
# print(datasets['train'].dataset)


# x,y = datasets['train'][0]
# print(x.shape, y)

#### Performing Train, Valid and Test Split

In [12]:
dataset_final = {}
datasets_created_trial_1 = train_val_dataset(dataset,val_split=0.10,generate_small=False)
datasets_created_trial_2 = train_val_dataset(datasets_created_trial_1['train'],val_split=0.25,generate_small=False)

dataset_final['test'] = datasets_created_trial_1['test']
dataset_final['train'] = datasets_created_trial_2['train']
dataset_final['val'] = datasets_created_trial_2['test']

print("Dataset set size {}".format(len(dataset)))
print("Train set size {}".format(len(dataset_final['train'])))
print("Validation set size {}".format(len(dataset_final['val'])))
print("Test set size {}".format(len(dataset_final['test'])))

Dataset set size 10615
Train set size 7164
Validation set size 2389
Test set size 1062


In [13]:
# for i in range(len(datasets['train'])):
#     sample = dataset[i]

#     print(i, sample['image'].shape, sample['isup_grade'])

#     if i == 3:
#         plt.show()
#         break

#### Performing WeightedRandomSampling using link : https://towardsdatascience.com/pytorch-basics-sampling-samplers-2a0f29f0bf2a

Generate class weights and target_list 

In [14]:
from collections import defaultdict 

count_dict = defaultdict(int)
target_list = []

#Generate target_list of all labels and count dict of all classes

for i, (image, label) in enumerate(dataset_final['train']):    
    count_dict[label] += 1
    target_list.append(label)
    
#     print(i, image.shape,sample)    
#     if i== 3:
#         break


count_dict
print("Distribution of classes: \n", count_dict)

class_count = [i for i in count_dict.values()]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
print("Class weights : \n", class_weights)

Distribution of classes: 
 defaultdict(<class 'int'>, {0: 1955, 4: 834, 1: 1798, 2: 915, 5: 823, 3: 839})


tensor([0.0005, 0.0012, 0.0006, 0.0011, 0.0012, 0.0012])

In [15]:
target_list = torch.tensor(target_list)
target_list = target_list[torch.randperm(len(target_list))]

In [16]:
class_weights_all = class_weights[target_list]
class_weights_all

tensor([0.0012, 0.0012, 0.0005,  ..., 0.0005, 0.0012, 0.0012])

In [19]:
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

### Use gpu and float as dtype

In [20]:

USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    

base_model = torchvision.models.resnet18(pretrained=False)

base_model = base_model.to(device)

# list(base_model.children())



## Step 3: Create Model definition

In [21]:
def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )

class ResNetUNet(nn.Module):

    def __init__(self, n_class):
        super().__init__()
        
        self.base_model = torchvision.models.resnet18(pretrained=True)

        self.base_layers = list(base_model.children())                
        
        self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)        
        self.layer1_1x1 = convrelu(64, 64, 1, 0)       
        self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)        
        self.layer2_1x1 = convrelu(128, 128, 1, 0)  
        self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)        
        self.layer3_1x1 = convrelu(256, 256, 1, 0)  
        self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
        self.layer4_1x1 = convrelu(512, 512, 1, 0)  
        
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)
        
        self.conv_original_size0 = convrelu(3, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)
        
        self.conv_last = nn.Conv2d(64, n_class, 1)
        # MY experiment
        self.fc = nn.Linear(301056, 6)
        
    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)
        
        layer0 = self.layer0(input)            
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)        
        layer4 = self.layer4(layer3)
        
        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)
        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)
 
        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)

        x = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)

        x = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)
        
        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)        
        
        out = self.conv_last(x)        

        # MY experiment
        x = out.view(out.size(0), -1)
        x = self.fc(x)
        
        # x = F.adaptive_avg_pool2d(out, output_size=1)
        # print(x.size(),x)

        
        return x



## Use new_train_enhanced_images below for training on images glued ones

In [22]:
def eval_model_8x(model, val_loader, device):
    """Evaluation without the densecrf with the dice coefficient"""
    model.eval()

    n_val = len(val_loader)  # the number of batch
    tot = 0

    for batch in val_loader:
        imgs, true_label = batch[0], batch[1]
        imgs = imgs.to(device=device, dtype=torch.float32)
        true_label = true_label.to(device=device, dtype=torch.long)
        with torch.no_grad():
            pred_label = model(imgs)

        tot += F.cross_entropy(pred_label, true_label).item()

    model.train()
    return tot / n_val

In [23]:
def new_train_enhanced_images(model, device, epochs=5, batch_size=32, lr=0.001, val_percent=0.1, save_cp=True):
    
#     dataloaders = {x:DataLoader(datasets[x],batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train','val']}
    train_loader = DataLoader(dataset_final['train'],batch_size=32, sampler = weighted_sampler, num_workers=4, pin_memory=True)
    val_loader = DataLoader(dataset_final['val'],batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

#     x,y = next(iter(dataloaders['train']))
#     print(x.shape, y.shape)

    dataset_sizes = {x: len(dataset_final[x]) for x in dataset_final.keys()}
    
    
    
    # dataset = BasicDataset(csv_file=tiny_csv_path, data_dir=data_dir)
    # n_val = int(len(dataset) * val_percent)
    # n_train = len(dataset) - n_val
    # train, val = random_split(dataset, [n_train, n_val])
    # train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    # val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)
    # dataloaders = {'train': train_loader, 'val':val_loader}
    # dataset_sizes = {'train':n_train, 'val':n_val}

    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter('runs/experiment_3_gluedimages')
    global_step = 0

    # print(f'''Starting training:
    #     Epochs:          {epochs}
    #     Batch size:      {batch_size}
    #     Learning rate:   {lr}
    #     Training size:   {n_train}
    #     Validation size: {n_val}
    #     Checkpoints:     {save_cp}
    #     Device:          {device.type}
    # ''')

    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0


    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))

        model.train()

        epoch_loss = 0.00 
        epoch_accuracy = 0.00
        running_loss = 0.00
        running_corrects = 0.00 
        for i, batch in enumerate(train_loader) :
            
#             print('Epoch {}, Step{}'.format(epoch, i))
            
            imgs = batch[0].to(device, dtype=torch.float32)
        
#             print(imgs.size())
#             imgs = torch.reshape(imgs,(imgs.shape[0],imgs.shape[3],imgs.shape[1],imgs.shape[2])) #  inputs.reshape [N, C, W, H]

            true_label = batch[1].to(device=device, dtype=torch.long)

            logits = model(imgs)
            _, pred_label = torch.max(logits, 1)
            loss = criterion(logits, true_label)

            epoch_loss += loss.item()
            running_loss += loss.item() * imgs.size(0) # batch_size 
            running_corrects += torch.sum(pred_label == true_label.data)

            #tensorboard
            img_grid = torchvision.utils.make_grid(imgs)
            # writer.add_image("training Images grid" , img_grid)
            writer.add_scalar('Training Loss', loss.item(), global_step)

            if i %10 ==0: # print loss every 10th step
                print('Epoch {} , Train Step {}, Training loss: {}'.format(epoch, i , loss.item()))

            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_value_(model.parameters(), 0.1)
            optimizer.step()


            global_step += 1
#             if global_step % (len(dataset) // (10 * batch_size)) == 0:
                # for tag, value in model.named_parameters():
                #     tag = tag.replace('.', '/')
                #     writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                #     writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)
        val_score = eval_model_8x(model, val_loader, device)
        scheduler.step()
        print('Validation loss: {}'.format(val_score))

        #tensorboard
        writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)
        writer.add_scalar('Validation Loss', val_score, global_step)
        writer.add_images('images', imgs, global_step)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(' epoch {}, Loss: {:.4f} Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))
        writer.add_scalar("epoch loss", epoch_loss, epoch)
        writer.add_scalar("epoch acc", epoch_acc, epoch)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                print('Created checkpoint directory')
            except OSError:
                pass
            torch.save(model.state_dict(),
                       dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
            print(f'Checkpoint {epoch + 1} saved !')
    writer.close()

### Trying with 8.x image dataset

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# device='cpu'
num_class = 6

model = ResNetUNet(num_class).to(device)

# freeze backbone layers
# Comment out to finetune further
for l in model.base_layers:
    for param in l.parameters():
        param.requires_grad = False

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)        
        
# model = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs=1)

model = new_train_enhanced_images(model, device, epochs=10, batch_size=32, lr=0.001,val_percent=0.1, save_cp=True)

cuda
Epoch 0/9
Epoch 0 , Train Step 0, Training loss: 1.8152673244476318
Epoch 0 , Train Step 10, Training loss: 1.7545740604400635
Epoch 0 , Train Step 20, Training loss: 1.8226145505905151
Epoch 0 , Train Step 30, Training loss: 1.6562457084655762
Epoch 0 , Train Step 40, Training loss: 1.7126117944717407


### End of Training